# Part 2 - Consume messages from an Apache Kafka topic

In this notebook you will learn how to consume messages to an Apache Kafka topic

![Consume messages from an Apache Kafka Topic](../img/consume.png)

_If you haven't already created and setup your virtual environment, follow the steps from [0-setup.ipynb](0-setup.ipynb)_

## Create an Apache Kafka consumer

The next step is to configure the Kafka consumer and have it run while we produce messages.

In [5]:
# Import Python Modules
import json
import os

from confluent_kafka import DeserializingConsumer, KafkaError, KafkaException

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

KAFKA_SERVICE_URI = os.getenv("KAFKA_SERVICE_URI")


def json_serializer(msg, s_obj):
    return json.loads(msg.decode('ascii'))

conf = {
    'bootstrap.servers': KAFKA_SERVICE_URI,
    'client.id': 'myclient',
    'group.id': 'pizzaioli',
    'security.protocol': 'SSL',
    'ssl.ca.location': '../sslcerts/ca.pem',
    'ssl.certificate.location': '../sslcerts/service.cert',
    'ssl.key.location': '../sslcerts/service.key', 
    'value.deserializer': json_serializer,
    'key.deserializer': json_serializer
    }

consumer = DeserializingConsumer(conf)

running = True

try:
    consumer.subscribe(["pizzas"])

    while running:
        msg = consumer.poll(timeout=1.0)
        if msg is None: continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                 (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            print ("%d:%d: k=%s v=%s" % (
                msg.partition(),
                msg.offset(),
                msg.key(),
                msg.value()))
except KeyboardInterrupt:
    print ("Shutdown requested...exiting")
finally:
    # Close down consumer to commit final offsets.
    consumer.close()    

0:1: k={'id': 2} v={'id': 2, 'name': '👩 Adele', 'pizza': 'Hawaii 🍕+🍍+🥓'}
0:2: k={'id': 3} v={'id': 3, 'name': '👦 Mark', 'pizza': 'Choccolate 🍕+🍫'}
Shutdown requested...exiting


---
Read messages